In [403]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale as mms

In [2]:
def load_data(path,name):
    csv_path = os.path.join(path, name)
    return pd.read_csv(csv_path)

In [34]:
path = ""
name = "wine.data"
data = load_data(path,name)
print("Loading wine dataset ...")
data.head()

Loading wine dataset ...


,class,atr1,atr2,atr3,atr4,atr5,atr6,atr7,atr8,atr9,atr10,atr11,atr12,atr13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


# Pre-processing Data Set

In [16]:
print("Convert Categorical Output to One Hot Vector")
def categorical_output(): 
    Y = data["class"]
    Y = Y.values.reshape(-1,1)
    enc = preprocessing.OneHotEncoder(categories='auto')
    enc.fit(Y)
    Y = enc.transform(Y).toarray() #Converting in hot vectors
    return Y
Y = categorical_output()
print(Y.shape)

Convert Categorical Output to One Hot Vector
(178, 3)


In [296]:
print("Preparing inputs droping Labels and droping indexs")
def preprocessing_input():
    X = data
    X = X.drop("class", axis = 1)
    X = mms(X) #Scalling 
    return X
X = preprocessing_input()
print(X.shape)

Preparing inputs droping Labels and droping indexs
(178, 13)


# The RBF Neurons

In [331]:
# Choosing  the centers as the media of each atribute
def choosing_centers(X):
    index = [] # Each center is a vector that works as a prototype of each class
    labels = data["class"]
    pivote = labels[0]
    centers = []
    for row in range(0,len(labels)):
        if labels[row] != pivote:
            index.append(row)
            pivote = labels[row]
    index.append(len(labels))    
    # Calculading the media of each class 
    tmp = 0
    for i in index:
        print(tmp,",",i)
        center = X[tmp:i]
        center = np.mean(center, axis=0)
        center = np.reshape(center, (1,-1))
        centers.append(center)
        tmp = i     
    return centers
print('Choosing the center indexes:')
centers = choosing_centers(X)

Choosing the center indexes:
0 , 59
59 , 130
130 , 178


In [329]:
def gaussian(beta,X,center):
    return np.exp(-beta*((np.linalg.norm(X-center,axis=1))**2))

In [335]:
#Exist a phi p/c center, in this particular case there are three vectors as centers , then three phi's
def RBF_act_fun(beta,X,centers):    
    beta = 1 # Beta represents the amplitud of Gaussian function
    phis = np.zeros((X.shape[0],1))
    for center in centers:
        phi = gaussian(1,X,center)
        phi = np.reshape(phi, (-1,1))
        phis = np.concatenate((phis,phi), axis=1) # We stack every phi column-vector in a matrix of phi's 
    phis =np.delete(phis, 0, 1)  # We delete unuseful column of 0's
    return phis
print('Calculating the phi(s) with respect each example: ',phis.shape)
phis = RBF_act_fun(1,X,centers)

Calculating the phi(s) with respect each example:  (178, 3)


# Individuals Perceptrons: Output Weights

In [337]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [343]:
def initialize_with_zeros(n_y,n_x):
    w = np.zeros(shape=(n_y, n_x))
    b = 0
    return w, b

In [381]:
def accuracy(Y_pred, Y):
    contador = 0
    Y_pred = Y_pred.T
    Y = Y.T
    for i in range(Y_pred.shape[0]) :
        if np.array_equal(Y_pred[i],Y[i]):
            contador +=1
    accuracy = (contador*100.0)/Y_pred.shape[0]
    return accuracy

In [391]:
def propagate(w, b, X, Y, pred=False):
    m = X.shape[1]
    
    # FORWARD PROPAGATION 
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)  
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A)))  # compute cost based in sigmoid function
    
    # BACKWARD PROPAGATION 
    dw = (1 / m) * np.dot(X, (A - Y).T)
    db = (1 / m) * np.sum(A - Y)
    
    grads = {"dw": dw,
             "db": db}
    
    # Predicction
    if pred:
        # The max value is 1 , else 0
        A_temp = A.T
        Y_prediction = np.zeros_like(A_temp)
        Y_prediction[np.arange(len(A_temp)), A_temp.argmax(1)] = 1 # Convert [0.9,0.2,0.05] en [1,0,0]
        Y_prediction = Y_prediction.T
        return Y_prediction
    return grads, cost

In [392]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):    
    costs = []
    
    for i in range(num_iterations):       
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        w = w - learning_rate * dw  # need to broadcast
        b = b - learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [430]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=1, learning_rate=0.5, print_cost=False):
    
    # Initialize parameters with zeros     
    w, b = initialize_with_zeros(Y_train.shape[0],X_train.shape[0]) 
    
    # Gradient descent 
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples
    #Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = propagate(w, b, X_train, Y_train, pred=True)
    Y_prediction_test  = propagate(w, b, X_test, Y_test, pred=True)

    # Print train/test Errors    
    print("Train Accuracy: ",accuracy(Y_prediction_train, Y_train)) 
    print("Test  Accuracy: ",accuracy(Y_prediction_test, Y_test))

In [431]:
X_train, X_test, Y_train, Y_test = train_test_split(phis, Y,
                                                    stratify=Y,
                                                    test_size=0.25,
                                                    random_state=0)
model(X_train.T, Y_train.T, X_test.T, Y_test.T, num_iterations=100, learning_rate=0.59)
print("Los datos estan en Shuffle")

Train Accuracy:  95.48872180451127
Test  Accuracy:  95.55555555555556
Los datos estan en Shuffle
